### **Архитектура работы**

1. **Запрос пользователя**  
   - Пользователь формулирует текстовый запрос, описывающий его информационную потребность.

2. **Создание вектора запроса**  
   - Запрос преобразуется в векторное представление с помощью модели `intfloat/multilingual-e5-large-instruct`.  
   - Это векторное представление отражает семантическое содержание текста.  

3. **Поиск релевантных данных**  
   - Вектор запроса сравнивается с векторами в хранилище Faiss.  
   - Используются методы ближайших соседей (k-NN) для определения наиболее релевантных документов.  
   - Возвращаются ссылки на несколько наиболее близких документов.  

4. **Генерация ответа**  
   - Найденные документы передаются в языковую модель `qwen2:7b-instruct-q4_0`.  
   - Модель анализирует данные, извлекает ключевую информацию и формирует краткий ответ.  
   - Ответ учитывает контекст найденных данных и запрос пользователя.  

In [1]:
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from sentence_transformers import SentenceTransformer
from langchain_ollama.llms import OllamaLLM
from langchain.schema import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
import faiss


/Users/y1ov/miniconda3/envs/new_env/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
file_path = 'house_dataset.csv'

In [3]:
df = pd.read_csv(file_path)
df

,"Площадь, м2",Количество спален,Участок,Адрес,Телефоны,Описание,Цена,Можно с детьми/животными,Дополнительно,Ссылка на объявление
0,380.0,4.0,"12.0, сот.","Москва, д. Бачурино, 254, ш. Калужское (7 км д...",+79853345356,"Уютный, светлый и современный двухэтажный дом,...","25000.0 руб./ За день, Залог - 30000 руб.",NaN,"Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/306675289
1,500.0,9.0,"30.0, сот.","Москва, Дмитровское шоссе, 122Гк10, ш. Дмитров...",+79623641352,"Уличный, открытый бассейн с горячей водой и зи...","6000.0 руб./ За день, Залог - 7000 руб.",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/279568123
2,500.0,5.0,"15.0, сот.","Москва, Краснопахорское поселение, д. Романцев...",+79154560650,"# 177\n\nСдаем новый коттедж 500 кв.м. в КП "" ...","25000.0 руб./ За день, Залог - 20000 руб.","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/301776830
3,560.0,7.0,"20.0, сот.","Москва, Сосенское поселение, Филатов Луг-2 СНТ...",+79154560650,КОТТЕДЖ #84 \n\nСдается в аренду посуточно 4-...,"25000.0 руб./ За день, Залог - 20000 руб.","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/229484352
4,750.0,11.0,"30.0, сот.","Москва, Десеновское поселение, ш. Калужское (1...",+79855840197,#23505\n3 ОСНОВНЫХ ПРЕИМУЩЕСТВА:\n-вместительн...,"29000.0 руб./ За день, Залог - 15000 руб.","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/265389810
...,...,...,...,...,...,...,...,...,...,...
195,80.0,NaN,"1.0, сот.","Москва, № 55 кв-л, 15, ш. Калужское (29 км до ...",+74952235776,"Грядок.Нет находится в Новой Москве, вблизи де...","14280.0 руб./ За день, Залог - 10000 руб.",Можно с детьми,"Мебель в комнатах, Телевизор, Интернет",https://www.cian.ru/rent/suburban/310118442
196,80.0,NaN,"1.0, сот.","Москва, № 55 кв-л, 15, ш. Калужское (29 км до ...",+74952235776,"Грядок.Нет находится в Новой Москве, вблизи де...","14280.0 руб./ За день, Залог - 10000 руб.",Можно с детьми,"Мебель в комнатах, Телевизор, Интернет",https://www.cian.ru/rent/suburban/310112729
197,200.0,5.0,"8.0, сот.","Москва, Щербинка, улица Парковая, 32А, ш. Симф...",+79858042828,Сдается дом посуточно от собственника!!! Подхо...,"15000.0 руб./ За день, Залог - 12000 руб.",Можно с детьми,"Мебель на кухне, Баня, Душевая кабина, Стираль...",https://www.cian.ru/rent/suburban/310643953
198,420.0,7.0,"20.0, сот.","Москва, Первомайское поселение, ш. Калужское (...","+79645905898, +79651102869","СДАЁТСЯ коттедж на сутки, выходные, праздники,...","15000.0 руб./ За день, Залог - 5000 руб.",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/154210827


In [4]:
file_path = 'house_dataset.csv'
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [5]:
class SentenceTransformerEmbeddings:
    def __init__(self, model_name="intfloat/multilingual-e5-large-instruct"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, query):
        return self.model.encode([query], convert_to_tensor=False)[0]

    def embed_documents(self, docs):
        return self.model.encode(docs, convert_to_tensor=False)

embeddings = SentenceTransformerEmbeddings()

/Users/y1ov/miniconda3/envs/new_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
embedding_dim = len(embeddings.embed_query(" "))  
index = faiss.IndexFlatL2(embedding_dim)
vector_store = FAISS(
    embedding_function=embeddings.embed_query,  
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

texts = [doc.page_content for doc in docs]
vector_store.add_texts(texts)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['a3be2c5c-e61d-46c9-881a-0414ed9c8072',
 '37f6aaa5-b1a0-4d02-94bc-95fa0914a5da',
 'b4760079-7d90-432d-b73a-edb7f52bbcc6',
 '5a975285-5421-4d39-b659-084458e63d88',
 '69121138-986f-4f4d-b3f4-776324204ca4',
 '28780b51-14ad-4aa3-81e6-553f12740088',
 'ffa0c25f-43c3-4a87-9fa2-ed675cb907a7',
 '6f46ee62-0166-46c8-81df-84c1ea58de5a',
 'a77de5d1-7875-4bfe-9083-06fe81e4b077',
 'ce93f7fe-8715-4f01-a90e-8ed590707037',
 '71aa431b-f8f9-4b2a-957b-6d6dc86151ea',
 '3b213501-75dc-4396-9253-f16f9a79bc64',
 '8d15aba7-6c4c-4210-b682-cb308eee696c',
 'cc6604ce-8604-4539-9059-7e34b43e989a',
 '90e3edde-30fd-402b-99dd-9012981ca26b',
 'a2bd8f94-236d-49d2-87af-d17afd57627f',
 'bf446111-3d10-4331-b159-bfc2278b68a2',
 '957fb195-3dec-4c08-97c7-ea740a313119',
 'e0d03ed2-ada8-4b15-9997-157cf61eae5c',
 'bdfb4c21-0724-4b95-86d8-c27bcca958a8',
 '8c66a2d6-3250-4af8-adaf-aa6dfe221ec8',
 '6bcedc23-1402-47e5-b3d3-6bd3f864834c',
 'af7a893b-f90e-4354-8c41-de5c28144db9',
 '6eeb1b3f-5f16-42de-9d71-66ba3132e72a',
 '521a9176-da23-

In [7]:
model_name = "qwen2:7b-instruct-q4_0"  
model = OllamaLLM(model=model_name, num_ctx=1000, temperature=0.0, top_k=1, max_tokens=25)

system_message = SystemMessage(content="Ты бот-помощник. Отвечай только на русском языке! Предоставь все релевантные данные о доме! ОБЯЗАТЕЛЬНО в конце отправляй номер телефона для связи и ссылку на объявление! Если нет идеально подходящего варианта, то обязательно скинь даже не самый лучший вариант!")

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_message}"),
    ("human", "{input}"),
])

In [8]:
def create_ollama_chain(retriever, system_message, model):
    def answer_question(input_text):
        
        retrieved_docs = retriever.get_relevant_documents(input_text, k = 10)
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])

        messages = [
            system_message,
            HumanMessage(content=f"Контекст: {context}\n\nВопрос: {input_text}")
        ]

        response = model.invoke(messages)
        
        if isinstance(response, str):
            return response
        
        return response.get('content', "Не удалось получить ответ.")
    
    return answer_question


In [9]:
retriever = vector_store.as_retriever()
ollama_chain = create_ollama_chain(retriever, system_message, model)

In [10]:
question = "нужен дом для вечеринки"
answer = ollama_chain(question)
print("Ответ:", answer)

/var/folders/yj/4rxbxmm11s7dztdck28lx4xm0000gn/T/ipykernel_69688/2488210125.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(input_text, k = 10)


Ответ: Для проведения вечеринки вам могут подойти два дома, которые я нашел:

1. Дом с адресом: Москва, Троицк, улица 8-я Вишневая, 3, ш. Калужское (18 км до МКАД), ш. Киевское (25 км до МКАД). Цена аренды в будни составляет от 15000 рублей за день и в выходные - от 25000 рублей.

2. Другой дом с адресом: Москва, Троицк, улица 8-я Вишневая, 3, ш. Калужское (18 км до МКАД), ш. Киевское (25 км до МКАД). Цена аренды в будни составляет от 15000 рублей за день и в выходные - от 25000 рублей.

Оба дома могут вместить до 35 человек, у них есть необходимая посуда для сервировки. Также они располагают гостиной площадью 70 кв.м., где можно приготовить шашлыки и курить. 

Пожалуйста, учтите, что в обоих домах есть интернет, телевизор, стиральная машина, посудомоечная машина и холодильник.

Если вам нужен более подробный анализ или дополнительная информация о каждом из этих домов, пожалуйста, дайте мне знать.
